# Lab 4.2 Use generative Language Models as text generator

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

Most of you have used ChatGPT from OpenAI and noticed that it can respond to your input in a very natural way. ChatGPT is build on top of GPT (Generative Pretrained Transformer) which is a model trained to generate text given a preceding input, so-called prompt (Brown et al 2020). It can do this repetitively up to a certain length, likewise generating short stories.

Models such as ChatGPT, although having good performance, are by far too large model to work with locally. Therefore in this notebook, we will use smaller open source models such as Llama and Qwen which are publicly available on huggingface.co.

### References

Brown, T. B., Mann, B., Ryder, N., Subbiah, M., Kaplan, J., Dhariwal, P., Neelakantan, A., Shyam, P., Sastry, G., Askell, A., et al. Language models are few-shot learners. arXiv preprint arXiv:2005.14165, 2020.

OpenAI, 2023. GPT-4 Technical Report. arXiv:2303.08774

Llama: Touvron, Hugo, Thibaut Lavril, Gautier Izacard, Xavier Martinet, Marie-Anne Lachaux, Timothée Lacroix, Baptiste Rozière et al. "Llama: Open and efficient foundation language models." arXiv preprint arXiv:2302.13971 (2023). [The Llama 3 Herd of Models](https://scontent-ams2-1.xx.fbcdn.net/v/t39.2365-6/452387774_1036916434819166_4173978747091533306_n.pdf?_nc_cat=104&ccb=1-7&_nc_sid=3c67a6&_nc_ohc=t6egZJ8QdI4Q7kNvgEka7Z4&_nc_ht=scontent-ams2-1.xx&oh=00_AYBPnjp-CQn7YnUQU_P-yJATmlaN6oRuEHZ1VrXshBoBwQ&oe=66A6EB8D)

Qwen: Bai, Jinze, Shuai Bai, Yunfei Chu, Zeyu Cui, Kai Dang, Xiaodong Deng, Yang Fan et al. "Qwen technical report." arXiv preprint arXiv:2309.16609 (2023).

Generative LLMs are trained to respond to prompts. The prompt is given as a list of **dict** strings defining the role of the **system** (the model) and the input given by the **user**:

In [ ]:
prompt = [{"role": "system", "content": "Generate 10 alternative sentences by completing the next input text."},
          {"role": "user", "content": "Bach sat down at his organ and played"}]

The ```transformers``` module can also be used to create a pipeline to generate a text as a response to prompt:

```# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen3-1.7B-Base")
messages = [
    {"role": "user", "content": "Who are you?"},
]

response = pipe(messages)
print(response)
```

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen3-1.7B-Base")
messages = [
    {"role": "user", "content": "Who are you?"},
]

response = pipe(messages)
print(response)


Device set to use mps:0
/Users/piek/.pyenv/versions/3.10.11/lib/python3.10/site-packages/transformers/pytorch_utils.py:328: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


Instead of the pipeline modules, we will use another package ```ChatOllama``` from **langchain** that was also used for your chat before.

In [2]:
from langchain_ollama import ChatOllama

model="qwen3:1.7b"

chat = ChatOllama(
            model=model,
            temperature= 0.1,
        )

In [6]:
response = chat.invoke(prompt)

### We need to remove the <think></think> part from the output
end_of_think = response.content.find("</think>")
answer = response
if end_of_think>0:
    think = response.content[:end_of_think+8]
    answer = response.content[end_of_think+8:].replace("\n", "")

In [9]:
print("Thinking aloud", think)
print()
print("The final answer", answer)

Thinking aloud <think>
Okay, the user wants me to generate 10 alternative sentences by completing the sentence "Bach sat down at his organ and played". Let me start by understanding the original sentence. It's about Johann Sebastian Bach playing his organ. The key elements are Bach, sitting down, the organ, and playing.

I need to create different sentences that maintain the structure but vary the details. Maybe change the verb tense, the object, or the context. Let me think of different ways to describe what he played. Instead of "organ," maybe "harpsichord" or "piano." Also, could vary the action, like "composed" instead of "played," or "sung" instead of "played." 

Wait, the original sentence uses "played" as a verb. So alternatives could include different verbs. For example, "composed," "sung," "wrote," "performed," "played a piece," "played a solo," etc. Also, maybe different prepositions or adjectives. 

Let me brainstorm some possibilities:

1. Bach sat down at his organ and com

When we define the client, we can not only adjust the prompt but also the temperature. Temperatures closer to 1.0 make the model become more creative, closer to 1.0 it will follow the instructions more in a strict way. So let's try again with a high temperature.

In [24]:
chat = ChatOllama(
            model=model,
            temperature= 0.9,
        )

In [25]:
response = chat.invoke(prompt)

### We need to remove the <think></think> part from the output
end_of_think = response.content.find("</think>")
answer = response
if end_of_think>0:
    think = response.content[:end_of_think+8]
    answer = response.content[end_of_think+8:].replace("\n", "")

print(answer)

1. Bach sat down at his organ and played with a fervent dedication.  2. Bach sat down at his organ and played in a graceful, flowing manner.  3. Bach sat down at his organ and played with a deep sense of solemnity.  4. Bach sat down at his organ and played for a long, mesmerizing duration.  5. Bach sat down at his organ and played during a particularly challenging performance.  6. Bach sat down at his organ and played in the presence of his devoted students.  7. Bach sat down at his organ and played as he always had, with unwavering focus.  8. Bach sat down at his organ and played with a unique, personal touch.  9. Bach sat down at his organ and played in the style of his most iconic compositions.  10. Bach sat down at his organ and played with a heartwarming, joyful expression.


You can clearly see that it got more expressive.

Let us set the temperature back to 0.1 and try another one:

In [26]:
chat = ChatOllama(
            model=model,
            temperature= 0.1,
        )

prompt = [{"role": "system", "content": "Generate 10 alternative sentences by completing the next input text."},
          {"role": "user", "content": "Beethoven sat down at his organ and played"}]


response = chat.invoke(prompt)

### We need to remove the <think></think> part from the output
end_of_think = response.content.find("</think>")
answer = response
if end_of_think>0:
    think = response.content[:end_of_think+8]
    answer = response.content[end_of_think+8:].replace("\n", "")

print(answer)

1. Beethoven sat at his organ and played.  2. Beethoven seated at his organ and played.  3. Beethoven set down at his organ and played.  4. Beethoven started playing the organ.  5. Beethoven commenced playing the organ.  6. Beethoven played the organ as he sat down.  7. Beethoven played the organ while sitting down.  8. Beethoven played the organ and sat down.  9. Beethoven played the organ, then sat down.  10. Beethoven played the organ, after sitting down.


Let us switch the language of the input, as Qwen is a multilingual model:

In [27]:
prompt = [{"role": "system", "content": "Generate 10 alternative sentences by completing the next input text in Dutch."},
          {"role": "user", "content": "Beethoven ging achter zijn orgel zitten en speelde"}]


response = chat.invoke(prompt)

### We need to remove the <think></think> part from the output
end_of_think = response.content.find("</think>")
answer = response
if end_of_think>0:
    think = response.content[:end_of_think+8]
    answer = response.content[end_of_think+8:].replace("\n", "")

print(answer)

1. Beethoven ging achter zijn orgel zitten en speelde het werk.  2. Beethoven ging achter zijn orgel zitten en speelde het op.  3. Beethoven ging achter zijn orgel zitten en speelde het kantoor.  4. Beethoven ging achter zijn orgel zitten en speelde het licht.  5. Beethoven ging achter zijn orgel zitten en speelde het zachte.  6. Beethoven ging achter zijn orgel zitten en speelde het zacht.  7. Beethoven ging achter zijn orgel zitten en speelde het werk.  8. Beethoven ging achter zijn orgel zitten en speelde het op.  9. Beethoven ging achter zijn orgel zitten en speelde het werk.  10. Beethoven ging achter zijn orgel zitten en speelde het op.


Instructions in English, completions as Dutch. You can also give the instructions in Dutch if you want or try other languages. Which languages are covered by Llama and how well? Interesing result is that Llama varied the sitting and in a few cases the organ (orgel) but never specified what Beethoven is playing.

In [28]:
prompt = [{"role": "system", "content": "Genereer 10 alternatieve zinnen door de volgende text af te maken."},
          {"role": "user", "content": "Beethoven ging achter zijn orgel zitten en speelde"}]

response = chat.invoke(prompt)

### We need to remove the <think></think> part from the output
end_of_think = response.content.find("</think>")
answer = response
if end_of_think>0:
    think = response.content[:end_of_think+8]
    answer = response.content[end_of_think+8:].replace("\n", "")

print(answer)

1. Beethoven played his organ with a mix of emotion and technical skill.  2. Beethoven played the organ in a grand, dramatic style.  3. Beethoven played the organ to honor the memory of his father.  4. Beethoven played the organ while composing his symphonies.  5. Beethoven played the organ in a quiet, reflective mood.  6. Beethoven played the organ with great passion, captivating the audience.  7. Beethoven's first performance on the organ was a success.  8. Beethoven played the organ as a way to express his inner emotions.  9. Beethoven played the organ with a blend of technical mastery and artistic expression.  10. Beethoven played the organ in front of the audience, marking a historic moment in music.


So Dutch instructions do make a big difference as the quality went down drastically.

## End of notebook